In [142]:
from string import punctuation
from collections import Counter
from tqdm import tqdm
import json
import os

import gensim
import numpy as np
import pandas as pd

from yargy import rule, Parser, or_, and_
from yargy.predicates import gte, lte, caseless, normalized, dictionary, gram, is_capitalized

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/amazon-beauty-reviews/All_Beauty.json
/kaggle/input/amazon-fashion-meta/meta_All_Beauty.json


In [138]:
!pip install yargy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.1/41.1 kB 747.4 kB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 13.4 MB/s eta 0:00:0000:0100:01


## Описание

Выберите корпус отзывов на товары одной из категорий Amazon:
http://jmcauley.ucsd.edu/data/amazon/


Допустим, что вам нужно подготовить аналитический отчет по этим отзывам — например, для производителя нового продукта этой категории. Для этого будем искать упоминания товаров в отзывах (будем считать их NE). Учтите, что упоминание может выглядеть не только как "Iphone 10", но и как "модель", "телефон" и т.п.

**Важное замечание**: в задании приводятся примеры решений, вы можете их использовать!

In [45]:
def str2dict(s: str):
    return json.loads(s[0:-1])


path_text = "/kaggle/input/amazon-beauty-reviews/All_Beauty.json"
df_text = pd.DataFrame([str2dict(i) for i in open(path_text)])

In [46]:
df_text.tail()

,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,style,image
371340,1.0,True,"07 20, 2017",A202DCI7TV1022,B01HJEGTYK,Sam,It was awful. It was super frizzy and I tried ...,It was super frizzy and I tried to comb it and...,1500508800,NaN,NaN,NaN
371341,5.0,True,"03 16, 2017",A3FSOR5IJOFIBE,B01HJEGTYK,TYW,I was skeptical about buying this. Worried it...,Awesome,1489622400,34,NaN,NaN
371342,5.0,True,"03 1, 2017",A1B5DK6CTP2P24,B01HJEGTYK,Norma Jennings,Makes me look good fast.,Five Stars,1488326400,46,NaN,NaN
371343,2.0,True,"02 21, 2017",A23OUYS5IRMJS9,B01HJEGTYK,Lee,Way lighter than photo\nNot mix blend of color...,Ok but color way off and volume as well,1487635200,NaN,NaN,NaN
371344,2.0,True,"12 15, 2016",A24KQ9RVU81L87,B01HJEGTYK,Season341,No return instructions/phone # in packaging. ...,Might return for a replacement if I could.,1481760000,NaN,NaN,NaN


In [47]:
df_text['review'] = df_text['reviewText'] + ' ' + df_text['summary']
df_text = df_text[['review']]

In [48]:
df_text.shape

(371345, 1)

In [19]:
path_meta = "/kaggle/input/amazon-fashion-meta/meta_All_Beauty.json"
df_meta = pd.DataFrame([str2dict(i) for i in open(path_meta)])
df_meta.head()

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,details,main_cat,similar_item,date,price,asin,imageURL,imageURLHighRes
0,[],,[Loud 'N Clear Personal Sound Amplifier allows...,,Loud 'N Clear&trade; Personal Sound Amplifier,[],,idea village,[],"2,938,573 in Beauty & Personal Care (",[],{'ASIN: ': '6546546450'},All Beauty,,,,6546546450,[],[]
1,[],,[No7 Lift & Luminate Triple Action Serum 50ml ...,,No7 Lift &amp; Luminate Triple Action Serum 50...,"[B01E7LCSL6, B008X5RVME]",,,[],"872,854 in Beauty & Personal Care (",[],"{'Shipping Weight:': '0.3 ounces (', 'ASIN: ':...",All Beauty,"class=""a-bordered a-horizontal-stripes a-spa...",,$44.99,7178680776,[],[]
2,[],,[No7 Stay Perfect Foundation now stays perfect...,,No7 Stay Perfect Foundation Cool Vanilla by No7,[],,No7,[],"956,696 in Beauty & Personal Care (","[B01B8BR0O8, B01B8BR0NO, B014MHXXM8]","{'Shipping Weight:': '3.5 ounces (', 'ASIN: ':...",All Beauty,,,$28.76,7250468162,[],[]
3,[],,[],,Wella Koleston Perfect Hair Colour 44/44 Mediu...,[B0041PBXX8],,,[],"1,870,258 in Beauty & Personal Care (",[],"{' Item Weight: ': '1.76 ounces', 'Sh...",All Beauty,,,,7367905066,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...
4,[],,[Lacto Calamine Skin Balance Daily Nourishing ...,,Lacto Calamine Skin Balance Oil control 120 ml...,[],,Pirmal Healthcare,[],"67,701 in Beauty & Personal Care (","[3254895630, B007VL1D9S, B00EH9A0RI, B0773MBG4...","{'Shipping Weight:': '12 ounces (', 'ASIN: ': ...",All Beauty,,,$12.15,7414204790,[https://images-na.ssl-images-amazon.com/image...,[https://images-na.ssl-images-amazon.com/image...


In [20]:
brands = set(df_meta[df_meta['brand'] != '']['brand'].tolist())

In [22]:
len(brands)

7862

## 1. ✅ (3 балла) Предложите 3 способа найти упоминания товаров в отзывах. 
Например, использовать bootstrapping: составить шаблоны вида "холодильник XXX", найти все соответствующие n-граммы и выделить из них называние товара.
Могут помочь заголовки и дополнительные данные с Amazon (Metadata [здесь](https://nijianmo.github.io/amazon/index.html)). Какие данные необходимы для каждого из способов? Какие есть достоинства/недостатки?

    1.  Dictionary-based подход
    
    Достоинства: простота (типично для бейслайна), хорошо работает со статичными типами entities, мало неоднозначности, скорость

    Недостатки: нужны тексты стандартного вида, область должна быть предметно ограничена, плохо работает с динамичными классами entities, словарь надо делать чем больше, тем лучше (возможно придется еще делать fuzzy search)
    
    2. Rule-based подход (например, Yargy)
    
    Достоинства: легко формализовать сущности, мало неоднозначности, скорость, в правилаъ можно комбинировать точные вхождения и pos
    
    Недостатки: времязатратность, нужны тексты стандартного вида, область должна быть предметно ограничена
    
    3. Classification-based
    
    Достоинства: определяет то, что не было на трейне; отлично работает с single-token entities, 
    
    Недостатки: проблемы с multi-token entities, требует большое количество размеченный training data (нужно создавать и аннотировать корпус),  
    
    2. SpaCy
    
    Достоинства: мультиязычность, сокрость
    
    Недостатки: нет

## 2. (2 балла) Реализуйте один из предложенных вами способов.

Примеры в качестве подсказки (можно использовать один из них): 
- написать правила с помощью [natasha/yargy](https://github.com/natasha/yargy)
- составить мини-словарь сущностей/дескрипторов, расширить с помощью эмбеддингов (например, word2vec)

**Обучаем w2v**

Обычно говорят _beauty product_, поэтому будем искать сущности, похожие на слово _product_.

In [109]:
def preprocessing(text):
    text = text.lower()
    for punct in punctuation:
        text = text.replace(punct, ' ')
    text = text.split()
    return text

In [110]:
corpora = [preprocessing(t) for t in df_text['review'].tolist() if type(t) != float]

In [115]:
model = gensim.models.Word2Vec(corpora, window=3, sg=0)

In [117]:
model.wv.most_similar('product', topn=15)

[('item', 0.8064481019973755),
 ('stuff', 0.7220137715339661),
 ('device', 0.6982458829879761),
 ('serum', 0.6399807333946228),
 ('deodorant', 0.5975395441055298),
 ('company', 0.5917617082595825),
 ('seller', 0.5757859349250793),
 ('cream', 0.5697977542877197),
 ('unit', 0.5678675770759583),
 ('moisturizer', 0.5619603991508484),
 ('toner', 0.5512058138847351),
 ('wig', 0.5422056913375854),
 ('steamer', 0.5350231528282166),
 ('model', 0.5335851907730103),
 ('waterpik', 0.5298057794570923)]

Запоминаем самые частотные слова похожие на _product_.

In [125]:
bow = []
for words in tqdm(corpora):
    bow.extend(words)
word_counter = Counter(bow)

100%|██████████| 370756/370756 [00:00<00:00, 894753.32it/s]


In [134]:
print('product:', word_counts['product'])
for i in model.wv.most_similar('product', topn=10):
    print(i[0], ': ', word_counter[i[0]], sep='')

product: 126014
item: 10416
stuff: 16104
device: 2097
serum: 3761
deodorant: 6164
company: 4418
seller: 4266
cream: 14788
unit: 5026
moisturizer: 4820


В п. 5 нам нужно будет анализировать 5 товаров, так что помимо _product_ запоминаем _item, stuff, cream, unit, moisturizer_.

In [136]:
top_words = ['product', 'item', 'stuff', 'cream', 'unit', 'moisturizer']

In [ ]:
TITLE = dictionary({
    'product',
    'item',
    'stuff',
    'cream',
    'unit',
    'moisturizer'
})

VUZ = rule(
    gram('ADJF'),  # теги pymorphy
    gram('ADJF'),
    TITLE
)

## 3. (1 балл) Соберите n-граммы с полученными сущностями (NE + левый сосед / NE + правый сосед)

## 4. (3 балла) Ранжируйте n-граммы с помощью 3 коллокационных метрик (t-score, PMI и т.д.). 
Не забудьте про частотный фильтр / сглаживание. Выберите лучший результат (какая метрика  ранжирует выше коллокации, подходящие для отчёта).

## 5. (1 балл) Сгруппируйте полученные коллокации по NE, выведите примеры для 5 товаров.
Должны получиться примерно такие группы:

```
watch 
--- 
stylish watch
good watches
great watch
love this watch
...
```

## 6. (2 балла) Бонус 
если придумаете способ объединить синонимичные упоминания (например, "Samsung Galaxy Watch", "watch", "smartwatch")